In [16]:
from ftplib import FTP
from datetime import datetime, timedelta
import tempfile
import csv
import os
import time

import pandas as pd

In [17]:
ftp_host = "**.***.***.**"    
ftp_user = "****"
ftp_password = "****"


ftp = FTP(ftp_host, ftp_user, ftp_password)
ftp.voidcmd("NOOP")
ftp.cwd('old')

'250 CWD command successful'

In [77]:
#Asia server

filenames = []
data = []
files = []


def get_dirs_ftp(folder=""):
    contents = ftp.nlst(folder)
    folders = []
    for item in contents:
        if "." not in item:
            folders.append(item)
            #print(item)
            for name, facts in ftp.mlsd(item):
                if facts['type'] == 'file': 
                    print(f'Found file {(item + "/" + name)}')
                    files.append({'file': item + "/" + name, 'modify': facts['modify']}) 
                
    return folders
def get_all_dirs_ftp(folder=""):
    dirs = []
    new_dirs = []
    new_dirs = get_dirs_ftp(folder)
    while len(new_dirs) > 0:
        for dir in new_dirs:
            dirs.append(dir)

        old_dirs = new_dirs[:]
        new_dirs = []
        for dir in old_dirs:
            for new_dir in get_dirs_ftp(dir):
                new_dirs.append(new_dir)
                #print(dir)
    
        
    dirs.sort()
    return dirs

allfiles = []
all_dirs = get_all_dirs_ftp()

Found file BZ/German_xPMbgl_BZ_F_2.txt
Found file CK/German_xPMbgl_CK_F_3.txt
Found file DH/German_xPMbgl_DH_F_4.txt
Found file ECJH/German_xPMbgl_ECJH_F_5.txt
Found file SZ/German_xPMbgl_SZ_F_6.txt
Found file TQ/German_xPMbgl_TQ_F_7.txt


error_perm: 501 Couldn't open the directory

In [22]:
#Eur server

files = []

#remove = ['/20220919', '/20220920', '/20220921', '/20220922', '/20220923', '/20220926', '/20220927', '/20221004', '/20221005', '/20221006', '/20221007', '/20221009', '/20221010', '/20221012', '/20221013', '/20221014', '/20221017', '/20221018', '/20221019', '/20221020', '/20221022', '/20221024', '/20221027', '/20221028', '/20221101', '/20221102', '/20221103', '/20221114', '/20221117', '/20221121', '/20221122', '/20221123', '/20221124', '/20221125', '/20221202', '/20221205']
remove = ['/Czech', '/French', '/German', '/Greek', '/Italian', '/Poland', '/Portuguese', '/Spanish']

def get_dirs_ftp(folder=""):
    #print('folder:',folder)
    contents = ftp.nlst(folder)
    
    absolute_path = [folder + "/"+ x for x in contents]
    folders = []
    
    for item, path in zip(contents, absolute_path):
        if "." not in item:
            folders.append(item)
               
            if path not in remove:
                print(path)
                for name, facts in ftp.mlsd(path):
                    if facts['type'] == 'file' and ".wav" in name: 
                       
                        print(f'Found file {(path + "/" + name)}')
                        files.append({'file': path + "/" + name, 'modify': facts['modify']})
            
    return folders

    
def get_all_dirs_ftp(folder=""):
    
    dirs = []
    new_dirs = []
    new_dirs = get_dirs_ftp(folder)
    
    while len(new_dirs) > 0:
        for dir in new_dirs:
            dirs.append(dir)

        old_dirs = new_dirs[:]
        new_dirs = []
        for directory in old_dirs:
            for new_dir in get_dirs_ftp(directory):
                new_dir = directory + "/" + new_dir 
                new_dirs.append(new_dir)
                
                
    dirs.sort()
    return dirs


allfiles = []
all_dirs = get_all_dirs_ftp()

'#Eur server\n\nfiles = []\n\n#remove = [\'/20220919\', \'/20220920\', \'/20220921\', \'/20220922\', \'/20220923\', \'/20220926\', \'/20220927\', \'/20221004\', \'/20221005\', \'/20221006\', \'/20221007\', \'/20221009\', \'/20221010\', \'/20221012\', \'/20221013\', \'/20221014\', \'/20221017\', \'/20221018\', \'/20221019\', \'/20221020\', \'/20221022\', \'/20221024\', \'/20221027\', \'/20221028\', \'/20221101\', \'/20221102\', \'/20221103\', \'/20221114\', \'/20221117\', \'/20221121\', \'/20221122\', \'/20221123\', \'/20221124\', \'/20221125\', \'/20221202\', \'/20221205\']\nremove = [\'/Czech\', \'/French\', \'/German\', \'/Greek\', \'/Italian\', \'/Poland\', \'/Portuguese\', \'/Spanish\']\n\ndef get_dirs_ftp(folder=""):\n    #print(\'folder:\',folder)\n    contents = ftp.nlst(folder)\n    \n    absolute_path = [folder + "/"+ x for x in contents]\n    folders = []\n    \n    for item, path in zip(contents, absolute_path):\n        if "." not in item:\n            folders.append(item)\

In [23]:
df = pd.DataFrame(files)
df.rename(columns = {'file':'original'}, inplace = True)

df

"df = pd.DataFrame(files)\ndf.rename(columns = {'file':'original'}, inplace = True)\n\ndf"

In [24]:
df['file1'] = df['original'].str.rsplit('/', n=1).str.get(0)
six_chars = df['original'].str.split("_").str[1]
df['combined'] = df['file1'] + "/" + six_chars
df

'df[\'file1\'] = df[\'original\'].str.rsplit(\'/\', n=1).str.get(0)\nsix_chars = df[\'original\'].str.split("_").str[1]\ndf[\'combined\'] = df[\'file1\'] + "/" + six_chars\ndf'

In [21]:
df.to_csv("D:/FTP_old.csv")